<a href="https://colab.research.google.com/github/lcnature/PSY291/blob/main/PSY291_Ch9_ttest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Chapter 9 t-test**

## replacing an assumed known standard deviation by an estimated standard deviation

When we don't know the standard deviation `σ` of a population, we can replace it by the estimate `s` based on a sample, and estimate standard error $s_M$ using `s`.

Let's look at how $s_M$ is distributed

For simplicity, let's assume a score `X` follows a standard normal distribution, but a researcher does not really know the standard deviation. Let's only assume that he/she knows the mean is 0.

How does the estimated **standard error** for samples of size 5 look like?


In [ ]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt

# Let's simulate drawing samples of 10 for 10000 times.

all_std_errs = []
# this creates an empty list

sample_size = 5

for iteration in range(10000):
  sample = norm.rvs(loc=0, scale=1, size=sample_size)
  standard_deviation = np.std(sample, ddof=1)
  # remember that when we estimate standard deviation from a sample, the
  # degree of freedom is sample size minus 1. This is instructed by providing
  # the argument ddof=1
  standard_error = standard_deviation / np.sqrt(sample_size)

  all_std_errs.append(standard_error)
  # we append a new item into the list

plt.hist(all_std_errs,bins=50)
plt.xlabel('standard error')
plt.ylabel('count')
plt.show()



## The distribution of `t`, an equivalent of `z` in the case when standard error needs to be estimated.

Similar to how we defined z-score for sample mean, we can now define t statistic for sample mean, with the only difference being the standard error is estimated from a sample.

$$t = \frac{M-μ}{s_M} = \frac{\sqrt{n}(M-μ)}{s} $$

Let's look at how `t`'s distribution differs from a normal distribution, when `s` is estimated.


In [ ]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt

# This block of code may take a minute to run
all_degree_of_freedom = np.array([1, 2, 5, 10, 20, 50])
all_sample_sizes = all_degree_of_freedom + 1
# degree of freedom for t-statistic is always sample size minus 1

all_colors = ['red','orange','yellow','green','blue','magenta']
for sample_size, color in zip(all_sample_sizes, all_colors):
  # what this does is that in each iteration of this "for" loop, we take
  # a sample_size from the first list, and a color from the second list, both
  # in sequential order. This allows us to plot each histogram with a different color.


  all_t = []
  for iteration in range(100000):
    sample = norm.rvs(loc=0, scale=1.0, size=sample_size)
    # for simplicity, we choose to simulate data from a standard normal distribution

    s = np.std(sample, ddof=1)
    # this is the estimated standard deviation of the sample

    se = s / np.sqrt(sample_size)
    # this is the estimated standard error

    t = (np.mean(sample) - 0) / se

    all_t.append(t)
    # we append the t-statistic estimated from each sample


  plt.hist(all_t, bins=np.arange(-6,6.1,0.1),
           histtype='step', density=True,ec=color)
  plt.xlabel('t-statistic')
  plt.ylabel('count')
  plt.xlim([-6,6])


# let's then compare these histograms against the standard normal distribution,
# which is how z-score should be distributed
x = np.arange(-6, 6.1, 0.1)
y = norm.pdf(x)
plt.plot(x, y, '--k')
plt.legend(['t: df={}'.format(df) for df in all_degree_of_freedom] + ['standard normal'])
plt.ylabel('density')
plt.show()




## Calculate effect size

### Cohen's d



In [ ]:
import numpy as np

scores = [39, 41, 43, 45, 46, 47, 50, 51, 52]
mu = 50

d = (np.mean(scores) - mu) / np.std(scores, ddof=1)
print('Cohen''s d for this sample based on a null hypothesis of mu=50:')
print(d)

### $r^2$: variance explainable by treatment effect

In [ ]:
import numpy as np

scores = np.array([39, 41, 43, 45, 46, 47, 50, 51, 52])
mu = 50

total_var = np.sum((scores-mu) ** 2)
var_residual = np.sum((scores-np.mean(scores)) ** 2)

r2 = 1 - var_residual / total_var
# This is equivalent to the equation in our lecture note

print('r2 is', r2)

## Calculate confidence interval

For the example above, we can use t-distribution implemented in scipy.stats to calculate confidence interval




In [ ]:
from scipy.stats import t as t_dist
# We rename the module t inside our code just in case we redefine t for something else

confidence_level = 0.95
t_edge_left = t_dist.ppf((1-confidence_level)/2, df=len(scores)-1)
# ppf is the inverse of cumulative distribution function.
# we first find the size of each tail on both sides of he 95% confidence interval
# of the t-distribution. Note that we need to specify degree of freedom, df

# Since t-distribution is symmetric
t_edge_right = - t_edge_left

# Remember we have to calculate standard error, not just standard deviation
se = np.std(scores, ddof=1) / np.sqrt(len(scores))

CI = [np.mean(scores) + se * t_edge_left, np.mean(scores) + se * t_edge_right]

print('95% CI:',CI)



## Doing t-test in the real world

Since t-test is used so frequently, it has been well implemented that we can run in just one single line of code!

We are testing against the null hypothesis that the mean of the population is 50.

So we need to tell the function of t-test this number, together with all the scores in our sample. That's all!

Check out the document [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_1samp.html)


In [ ]:
from scipy.stats import ttest_1samp
# ttest_1samp stands for one-sample t-test, which is what we learned today.
# We will learn about t-test between two samples in the next class.

mu = 50
# this is the mean of null hypothesis that we are testing against
# scores is the scores in our sample
result = ttest_1samp(scores, popmean=mu)
print(result)

# we can also calculate confidence interval for any confidence level:
# For example,
print('80% CI:',result.confidence_interval(confidence_level=0.8))




## Practice

Perform a one-tailed t-test to draw a conclusion of whether the following sample provides sufficient evidence that the population mean is larger than 0:

scores = [-2, -1, -1, 1, 2, 2, 4, 5, 6]

calculate the p-value, Cohen's d (against a hypothesized mean of 0), and confidence interval at confidence level of 95%